In [1]:
import sys
sys.path.append('..')  # 添加父文件夹到sys.path
sys.path.append('../src')  # 添加父文件夹到sys.path
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
# from Model import DRSNet, RSBU_CW
import math
import random
from myModel import *
from Dataset import *
import csv

class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()

    def forward(self, predictions, targets):
        errors = torch.abs((targets - predictions) / targets)
        mape = torch.mean(errors) * 100
        return mape
    
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(3739)  # random prime
    
# writer = SummaryWriter("./drsn_train_441_v1")
# 每次训练记得改一下模型保存的名字
train_data = torch.load('./data/train_p345_D1.pt')
validation_data = torch.load('./data/validation_p1_D1.pt')

# DataLoad
train_load = DataLoader(dataset=train_data,  # shuffle 是否打乱顺序 drop_last 是否丢掉余数
                        batch_size=32, shuffle=True, num_workers=2, drop_last=True)  # num_worker
test_load = DataLoader(dataset=validation_data,
                       batch_size=32, shuffle=True, num_workers=2, drop_last=True)

C:\anaconda3\envs\yolov8-rxx\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def TrainModel(specModel, model_name, model_name_):
    setup_seed(3739)  # random prime
    specModel = specModel.double()
    specModel = specModel.cuda()
    optimizer = torch.optim.Adam(specModel.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.7)
    loss_fn = MAPELoss()
    loss_fn = loss_fn.cuda()
    # 用来记录训练的次数
    total_train_step = 0
    # 用来记录测试的次数
    total_test_step = 0
    # 训练的轮数
    epoch = 200
    # min_loss
    min_loss = math.inf

    losses = []
    best_models = [{'loss': float('inf'), 'model': None} for _ in range(3)]

    for i in range(epoch):
        print("-------第{}轮训练开始-------".format(i + 1))
        # 训练步骤开始
        specModel.train()  # 存在特殊网络层时需要调用
        for data in train_load:
            spec, targets = data
            # spec = spec.unsqueeze(1)
            spec = spec.cuda()
            targets = targets.cuda()
            outputs = specModel(spec)
            loss = loss_fn(outputs, targets)
            # 优化器优化模型
            optimizer.zero_grad()  # 使用前清零
            loss.backward()

            # if total_train_step == 5:  # 加速进程用于检查整体是否跑通
            #     break

            optimizer.step()
            total_train_step = total_train_step + 1

        # 测试步骤开始        # 存在特殊网络层时需要调用
        scheduler.step()
        specModel.eval()
        total_test_loss = 0
        total_accuracy = 0
        total_test_mse = torch.zeros(1)
        total_test_mse = total_test_mse.cuda()
        with torch.no_grad():
            for data in test_load:
                spec, targets = data
                # spec = spec.unsqueeze(1)
                spec = spec.cuda()
                targets = targets.cuda()
                outputs = specModel(spec)
                loss = loss_fn(outputs, targets)
                total_test_loss = total_test_loss + loss.item()
                se = (targets - outputs)
                se = se * se
                se = se.mean(dim=0)
                total_test_mse = total_test_mse + se

        print("整体测试集上的loss:{}".format(total_test_loss))

        total_test_step = total_test_step + 1
        losses.append(total_test_loss)
        if total_test_loss < min_loss:
            torch.save(specModel, "./modelSave/e_" + model_name)
            min_loss = total_test_loss

    with open('./trainLog/losses_'+ model_name_ + '.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Iteration", "Loss"])  # 写入列名
        for i, loss in enumerate(losses, 1):
            writer.writerow([i, loss])

In [3]:
# model = DNN()
# model.cuda()
# model.double()
# x = torch.rand(6, 1, 8192)
# x = x.cuda()
# x = x.double()
# out = model(x)
# print(out.size())

In [4]:
specModel = DRSN_diffCBAM()
model_name = 'DRSN_diffCBAM_MAPE_Adam.pth'
model_name_ = 'DRSN_diffCBAM_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:2064.4467520003027
-------第2轮训练开始-------
整体测试集上的loss:2281.633730678409
-------第3轮训练开始-------
整体测试集上的loss:2326.4263220577363
-------第4轮训练开始-------
整体测试集上的loss:1538.6288534619146
-------第5轮训练开始-------
整体测试集上的loss:1877.2638610157835
-------第6轮训练开始-------
整体测试集上的loss:2144.1612621378695
-------第7轮训练开始-------
整体测试集上的loss:2482.546693558126
-------第8轮训练开始-------
整体测试集上的loss:1666.2346792368915
-------第9轮训练开始-------
整体测试集上的loss:1686.9450362562488
-------第10轮训练开始-------
整体测试集上的loss:1811.0455959260719
-------第11轮训练开始-------
整体测试集上的loss:2107.309698674048
-------第12轮训练开始-------
整体测试集上的loss:1854.554744979524
-------第13轮训练开始-------
整体测试集上的loss:1843.3932849800017
-------第14轮训练开始-------
整体测试集上的loss:1057.5462415883023
-------第15轮训练开始-------
整体测试集上的loss:1444.9628315467444
-------第16轮训练开始-------
整体测试集上的loss:1379.1005443726017
-------第17轮训练开始-------
整体测试集上的loss:2050.6492952290077
-------第18轮训练开始-------
整体测试集上的loss:1124.9413397766566
-------第19轮训练开始-------
整体测试集上的loss:1163.3

整体测试集上的loss:1482.2085864530275
-------第154轮训练开始-------
整体测试集上的loss:1600.9861832774275
-------第155轮训练开始-------
整体测试集上的loss:1654.1393976987981
-------第156轮训练开始-------
整体测试集上的loss:1665.6854767536404
-------第157轮训练开始-------
整体测试集上的loss:1908.3244639583556
-------第158轮训练开始-------
整体测试集上的loss:1778.7603642338436
-------第159轮训练开始-------
整体测试集上的loss:1816.4771411694512
-------第160轮训练开始-------
整体测试集上的loss:1746.09873332815
-------第161轮训练开始-------
整体测试集上的loss:1450.773742649716
-------第162轮训练开始-------
整体测试集上的loss:1449.6467382613162
-------第163轮训练开始-------
整体测试集上的loss:1732.8391843042016
-------第164轮训练开始-------
整体测试集上的loss:1704.0648309885717
-------第165轮训练开始-------
整体测试集上的loss:1795.440379785414
-------第166轮训练开始-------
整体测试集上的loss:1663.0200071755057
-------第167轮训练开始-------
整体测试集上的loss:1873.7528899318359
-------第168轮训练开始-------
整体测试集上的loss:1957.3198673048596
-------第169轮训练开始-------
整体测试集上的loss:1788.0927344301072
-------第170轮训练开始-------
整体测试集上的loss:1847.2802255122167
-------第171轮训练开始-------
整体测试集上的loss:18

In [5]:
specModel = DRSN_1D_CBAM_v2()
model_name = 'DRSN_1D_CBAM_v2_MAPE_Adam.pth'
model_name_ = 'DRSN_1D_CBAM_v2_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:2447.1067553491985
-------第2轮训练开始-------
整体测试集上的loss:1764.095105441048
-------第3轮训练开始-------
整体测试集上的loss:1559.3824233362077
-------第4轮训练开始-------
整体测试集上的loss:2003.2360094899468
-------第5轮训练开始-------
整体测试集上的loss:1399.673600070417
-------第6轮训练开始-------
整体测试集上的loss:1394.7486283602077
-------第7轮训练开始-------
整体测试集上的loss:1555.768449240835
-------第8轮训练开始-------
整体测试集上的loss:1343.3884698636043
-------第9轮训练开始-------
整体测试集上的loss:1376.1737465232209
-------第10轮训练开始-------
整体测试集上的loss:1563.682514347695
-------第11轮训练开始-------
整体测试集上的loss:1701.5282551800203
-------第12轮训练开始-------
整体测试集上的loss:1383.2025048352598
-------第13轮训练开始-------
整体测试集上的loss:1478.166172013567
-------第14轮训练开始-------
整体测试集上的loss:1678.206707324494
-------第15轮训练开始-------
整体测试集上的loss:2006.2422252337694
-------第16轮训练开始-------
整体测试集上的loss:1637.1888150341374
-------第17轮训练开始-------
整体测试集上的loss:2078.0585507239875
-------第18轮训练开始-------
整体测试集上的loss:1541.0155711233376
-------第19轮训练开始-------
整体测试集上的loss:1717.941

整体测试集上的loss:2242.2853038765397
-------第154轮训练开始-------
整体测试集上的loss:2179.9287550692497
-------第155轮训练开始-------
整体测试集上的loss:2252.034641446561
-------第156轮训练开始-------
整体测试集上的loss:2070.5537091946826
-------第157轮训练开始-------
整体测试集上的loss:2205.9641975753916
-------第158轮训练开始-------
整体测试集上的loss:2204.3637674130796
-------第159轮训练开始-------
整体测试集上的loss:2203.6735996080424
-------第160轮训练开始-------
整体测试集上的loss:2075.500651144596
-------第161轮训练开始-------
整体测试集上的loss:2207.244677892599
-------第162轮训练开始-------
整体测试集上的loss:2256.968268094644
-------第163轮训练开始-------
整体测试集上的loss:2277.7526014493164
-------第164轮训练开始-------
整体测试集上的loss:2286.2568187978086
-------第165轮训练开始-------
整体测试集上的loss:2190.4329835293015
-------第166轮训练开始-------
整体测试集上的loss:2417.317170302925
-------第167轮训练开始-------
整体测试集上的loss:2241.4960032753656
-------第168轮训练开始-------
整体测试集上的loss:2156.8154224068057
-------第169轮训练开始-------
整体测试集上的loss:2085.4473466242857
-------第170轮训练开始-------
整体测试集上的loss:2059.8906933283693
-------第171轮训练开始-------
整体测试集上的loss:217

In [3]:
train_data = torch.load('./data/train_p345.pt')
validation_data = torch.load('./data/validation_p1.pt')

# DataLoad
train_load = DataLoader(dataset=train_data,  # shuffle 是否打乱顺序 drop_last 是否丢掉余数
                        batch_size=32, shuffle=True, num_workers=2, drop_last=True)  # num_worker
test_load = DataLoader(dataset=validation_data,
                       batch_size=32, shuffle=True, num_workers=2, drop_last=True)

In [4]:
specModel = DRSN()
model_name = 'DRSN_MAPE_Adam.pth'
model_name_ = 'DRSN_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:5889.666210178595
-------第2轮训练开始-------
整体测试集上的loss:2909.6238999398884
-------第3轮训练开始-------
整体测试集上的loss:2198.8803249086095
-------第4轮训练开始-------
整体测试集上的loss:1665.959231730894
-------第5轮训练开始-------
整体测试集上的loss:1313.9836549799186
-------第6轮训练开始-------
整体测试集上的loss:1340.1392440306793
-------第7轮训练开始-------
整体测试集上的loss:1319.7420608094742
-------第8轮训练开始-------
整体测试集上的loss:1509.924471391441
-------第9轮训练开始-------
整体测试集上的loss:1695.0963885701133
-------第10轮训练开始-------
整体测试集上的loss:828.7720371179297
-------第11轮训练开始-------
整体测试集上的loss:1535.4707569628986
-------第12轮训练开始-------
整体测试集上的loss:2215.812244630895
-------第13轮训练开始-------
整体测试集上的loss:1544.412039264827
-------第14轮训练开始-------
整体测试集上的loss:1216.8327091379438
-------第15轮训练开始-------
整体测试集上的loss:1659.8732083371262
-------第16轮训练开始-------
整体测试集上的loss:1104.8770840009036
-------第17轮训练开始-------
整体测试集上的loss:1555.7497062439513
-------第18轮训练开始-------
整体测试集上的loss:1230.103572218998
-------第19轮训练开始-------
整体测试集上的loss:1344.3512

整体测试集上的loss:1656.3675243811883
-------第154轮训练开始-------
整体测试集上的loss:1385.0402005767805
-------第155轮训练开始-------
整体测试集上的loss:1670.7188297049347
-------第156轮训练开始-------
整体测试集上的loss:1504.4200283826412
-------第157轮训练开始-------
整体测试集上的loss:1501.3736508211593
-------第158轮训练开始-------
整体测试集上的loss:1649.7570288110255
-------第159轮训练开始-------
整体测试集上的loss:1773.8990284601887
-------第160轮训练开始-------
整体测试集上的loss:1682.1542091319307
-------第161轮训练开始-------
整体测试集上的loss:1628.1233931240306
-------第162轮训练开始-------
整体测试集上的loss:1233.6323425165385
-------第163轮训练开始-------
整体测试集上的loss:1392.999881471625
-------第164轮训练开始-------
整体测试集上的loss:1735.5449655746036
-------第165轮训练开始-------
整体测试集上的loss:1428.0198539508112
-------第166轮训练开始-------
整体测试集上的loss:1624.2696032753217
-------第167轮训练开始-------
整体测试集上的loss:1645.5361910556053
-------第168轮训练开始-------
整体测试集上的loss:1626.2024623234872
-------第169轮训练开始-------
整体测试集上的loss:1595.159498153729
-------第170轮训练开始-------
整体测试集上的loss:1615.9369422958582
-------第171轮训练开始-------
整体测试集上的loss:

In [8]:
specModel = ResNet()
model_name = 'ResNet_MAPE_Adam.pth'
model_name_ = 'ResNet_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:12116.33448588712
-------第2轮训练开始-------
整体测试集上的loss:5307.458266653292
-------第3轮训练开始-------
整体测试集上的loss:2072.074299931413
-------第4轮训练开始-------
整体测试集上的loss:1724.2988271179154
-------第5轮训练开始-------
整体测试集上的loss:1925.9370599030624
-------第6轮训练开始-------
整体测试集上的loss:1838.2628430671462
-------第7轮训练开始-------
整体测试集上的loss:1382.5609365799505
-------第8轮训练开始-------
整体测试集上的loss:1383.4937398065008
-------第9轮训练开始-------
整体测试集上的loss:1626.46628688369
-------第10轮训练开始-------
整体测试集上的loss:1442.3387207152005
-------第11轮训练开始-------
整体测试集上的loss:1264.9933632265852
-------第12轮训练开始-------
整体测试集上的loss:2049.8847233831716
-------第13轮训练开始-------
整体测试集上的loss:1793.3078064177364
-------第14轮训练开始-------
整体测试集上的loss:1195.1170925578183
-------第15轮训练开始-------
整体测试集上的loss:1567.7751862861242
-------第16轮训练开始-------
整体测试集上的loss:1363.8435707634787
-------第17轮训练开始-------
整体测试集上的loss:2103.094294448901
-------第18轮训练开始-------
整体测试集上的loss:1220.5616497489177
-------第19轮训练开始-------
整体测试集上的loss:1768.157

整体测试集上的loss:1444.8689711874897
-------第154轮训练开始-------
整体测试集上的loss:1438.6420522783606
-------第155轮训练开始-------
整体测试集上的loss:1391.5203823237646
-------第156轮训练开始-------
整体测试集上的loss:1465.5192306921529
-------第157轮训练开始-------
整体测试集上的loss:1454.6299148531173
-------第158轮训练开始-------
整体测试集上的loss:1457.2152226851676
-------第159轮训练开始-------
整体测试集上的loss:1980.0354451557907
-------第160轮训练开始-------
整体测试集上的loss:1378.6203933165011
-------第161轮训练开始-------
整体测试集上的loss:1181.303034352978
-------第162轮训练开始-------
整体测试集上的loss:1375.439847740029
-------第163轮训练开始-------
整体测试集上的loss:1415.4504853165051
-------第164轮训练开始-------
整体测试集上的loss:1380.5741292076407
-------第165轮训练开始-------
整体测试集上的loss:1282.915055132432
-------第166轮训练开始-------
整体测试集上的loss:1667.1090791340573
-------第167轮训练开始-------
整体测试集上的loss:1469.5664033141709
-------第168轮训练开始-------
整体测试集上的loss:1082.1624015328
-------第169轮训练开始-------
整体测试集上的loss:1518.379518940241
-------第170轮训练开始-------
整体测试集上的loss:1471.074536449385
-------第171轮训练开始-------
整体测试集上的loss:1417.2

In [5]:
specModel = CNN()
model_name = 'CNN_MAPE_Adam.pth'
model_name_ = 'CNN_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:17377.83935713112
-------第2轮训练开始-------
整体测试集上的loss:3163.2254341913704
-------第3轮训练开始-------
整体测试集上的loss:4760.959944658402
-------第4轮训练开始-------
整体测试集上的loss:12543.703847371049
-------第5轮训练开始-------
整体测试集上的loss:3058.0351812658305
-------第6轮训练开始-------
整体测试集上的loss:3374.2357634623736
-------第7轮训练开始-------
整体测试集上的loss:3309.809752676
-------第8轮训练开始-------
整体测试集上的loss:2905.2263649442707
-------第9轮训练开始-------
整体测试集上的loss:3071.4132615204417
-------第10轮训练开始-------
整体测试集上的loss:1926.9116603286811
-------第11轮训练开始-------
整体测试集上的loss:1584.7093927058581
-------第12轮训练开始-------
整体测试集上的loss:2192.9592457549084
-------第13轮训练开始-------
整体测试集上的loss:1706.3267520161176
-------第14轮训练开始-------
整体测试集上的loss:1839.2208107785325
-------第15轮训练开始-------
整体测试集上的loss:1694.2773875711014
-------第16轮训练开始-------
整体测试集上的loss:1850.24465005551
-------第17轮训练开始-------
整体测试集上的loss:1800.2645317879933
-------第18轮训练开始-------
整体测试集上的loss:1690.8248582472984
-------第19轮训练开始-------
整体测试集上的loss:1873.66226

整体测试集上的loss:1432.6706475419128
-------第154轮训练开始-------
整体测试集上的loss:1546.5749449043547
-------第155轮训练开始-------
整体测试集上的loss:1714.416339897009
-------第156轮训练开始-------
整体测试集上的loss:1494.3703141135186
-------第157轮训练开始-------
整体测试集上的loss:1656.101305743018
-------第158轮训练开始-------
整体测试集上的loss:1463.6461241956229
-------第159轮训练开始-------
整体测试集上的loss:1484.9532381318631
-------第160轮训练开始-------
整体测试集上的loss:1329.8714981836551
-------第161轮训练开始-------
整体测试集上的loss:1604.0139450167871
-------第162轮训练开始-------
整体测试集上的loss:1523.1890196281438
-------第163轮训练开始-------
整体测试集上的loss:1514.7555066660323
-------第164轮训练开始-------
整体测试集上的loss:1678.0032284197457
-------第165轮训练开始-------
整体测试集上的loss:1559.8172265939693
-------第166轮训练开始-------
整体测试集上的loss:1332.776984369609
-------第167轮训练开始-------
整体测试集上的loss:1607.9309963044193
-------第168轮训练开始-------
整体测试集上的loss:1558.4922603788596
-------第169轮训练开始-------
整体测试集上的loss:1372.271137434746
-------第170轮训练开始-------
整体测试集上的loss:1338.5279698897164
-------第171轮训练开始-------
整体测试集上的loss:15

In [10]:
specModel = DNN()
model_name = 'DNN_MAPE_Adam.pth'
model_name_ = 'DNN_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:3306.4746105896124
-------第2轮训练开始-------
整体测试集上的loss:2579.5066816759727
-------第3轮训练开始-------
整体测试集上的loss:3178.9698810689442
-------第4轮训练开始-------
整体测试集上的loss:3001.6011812008664
-------第5轮训练开始-------
整体测试集上的loss:1857.2438541711053
-------第6轮训练开始-------
整体测试集上的loss:2107.771922029176
-------第7轮训练开始-------
整体测试集上的loss:2369.1273736514704
-------第8轮训练开始-------
整体测试集上的loss:2476.4806165638997
-------第9轮训练开始-------
整体测试集上的loss:2001.493613167909
-------第10轮训练开始-------
整体测试集上的loss:1836.501369415645
-------第11轮训练开始-------
整体测试集上的loss:2028.500340163342
-------第12轮训练开始-------
整体测试集上的loss:2146.5026314944093
-------第13轮训练开始-------
整体测试集上的loss:2002.8854851784308
-------第14轮训练开始-------
整体测试集上的loss:2107.4690310183673
-------第15轮训练开始-------
整体测试集上的loss:2005.9069051961194
-------第16轮训练开始-------
整体测试集上的loss:1902.2193517066607
-------第17轮训练开始-------
整体测试集上的loss:1849.6558939885417
-------第18轮训练开始-------
整体测试集上的loss:2064.038989052197
-------第19轮训练开始-------
整体测试集上的loss:1999.25

整体测试集上的loss:1832.7684669290363
-------第154轮训练开始-------
整体测试集上的loss:1846.925629856435
-------第155轮训练开始-------
整体测试集上的loss:1836.2704063767283
-------第156轮训练开始-------
整体测试集上的loss:1854.7456026173063
-------第157轮训练开始-------
整体测试集上的loss:1830.5582185272683
-------第158轮训练开始-------
整体测试集上的loss:1841.677501024973
-------第159轮训练开始-------
整体测试集上的loss:1851.317714922583
-------第160轮训练开始-------
整体测试集上的loss:1848.073430227973
-------第161轮训练开始-------
整体测试集上的loss:1838.9844628563096
-------第162轮训练开始-------
整体测试集上的loss:1838.4618141869805
-------第163轮训练开始-------
整体测试集上的loss:1830.4060577952991
-------第164轮训练开始-------
整体测试集上的loss:1838.5656363732921
-------第165轮训练开始-------
整体测试集上的loss:1848.964021866308
-------第166轮训练开始-------
整体测试集上的loss:1848.6313663539938
-------第167轮训练开始-------
整体测试集上的loss:1837.9430221109615
-------第168轮训练开始-------
整体测试集上的loss:1833.0221508838063
-------第169轮训练开始-------
整体测试集上的loss:1843.8453472486178
-------第170轮训练开始-------
整体测试集上的loss:1832.058495319646
-------第171轮训练开始-------
整体测试集上的loss:1835

In [3]:
specModel = DRSN_diff()
model_name = 'DRSN_diff_MAPE_Adam.pth'
model_name_ = 'DRSN_diff_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:2032.3819445355907
-------第2轮训练开始-------
整体测试集上的loss:2948.1365927264974
-------第3轮训练开始-------
整体测试集上的loss:1912.1659543856838
-------第4轮训练开始-------
整体测试集上的loss:2003.8417188879323
-------第5轮训练开始-------
整体测试集上的loss:1808.540933644003
-------第6轮训练开始-------
整体测试集上的loss:1813.0823083493674
-------第7轮训练开始-------
整体测试集上的loss:2021.9352732501404
-------第8轮训练开始-------
整体测试集上的loss:1260.3210443494124
-------第9轮训练开始-------
整体测试集上的loss:1767.3531822805735
-------第10轮训练开始-------
整体测试集上的loss:1190.1913755571532
-------第11轮训练开始-------
整体测试集上的loss:1393.188655843118
-------第12轮训练开始-------
整体测试集上的loss:2342.1598689373573
-------第13轮训练开始-------
整体测试集上的loss:1537.43881680924
-------第14轮训练开始-------
整体测试集上的loss:1329.8808534436707
-------第15轮训练开始-------
整体测试集上的loss:1004.2173874104946
-------第16轮训练开始-------
整体测试集上的loss:1136.1755487431496
-------第17轮训练开始-------
整体测试集上的loss:1843.3806524222155
-------第18轮训练开始-------
整体测试集上的loss:1327.0872066885222
-------第19轮训练开始-------
整体测试集上的loss:1763.9

整体测试集上的loss:1290.20893334323
-------第154轮训练开始-------
整体测试集上的loss:1551.0791021997527
-------第155轮训练开始-------
整体测试集上的loss:1338.5748736147527
-------第156轮训练开始-------
整体测试集上的loss:1596.390791281881
-------第157轮训练开始-------
整体测试集上的loss:1826.0764014784904
-------第158轮训练开始-------
整体测试集上的loss:1712.9054287973445
-------第159轮训练开始-------
整体测试集上的loss:1850.4332270676184
-------第160轮训练开始-------
整体测试集上的loss:1560.8055117852898
-------第161轮训练开始-------
整体测试集上的loss:2094.705020579086
-------第162轮训练开始-------
整体测试集上的loss:1831.619134382731
-------第163轮训练开始-------
整体测试集上的loss:2108.731399544276
-------第164轮训练开始-------
整体测试集上的loss:1614.0616518841857
-------第165轮训练开始-------
整体测试集上的loss:1698.127437974105
-------第166轮训练开始-------
整体测试集上的loss:2085.119564770663
-------第167轮训练开始-------
整体测试集上的loss:1769.8686276504664
-------第168轮训练开始-------
整体测试集上的loss:1467.1548933739252
-------第169轮训练开始-------
整体测试集上的loss:1829.619562643188
-------第170轮训练开始-------
整体测试集上的loss:2018.8785675770368
-------第171轮训练开始-------
整体测试集上的loss:1858.70

In [4]:
train_data = torch.load('./data/train_p345.pt')
validation_data = torch.load('./data/validation_p1.pt')

# DataLoad
train_load = DataLoader(dataset=train_data,  # shuffle 是否打乱顺序 drop_last 是否丢掉余数
                        batch_size=32, shuffle=True, num_workers=2, drop_last=True)  # num_worker
test_load = DataLoader(dataset=validation_data,
                       batch_size=32, shuffle=True, num_workers=2, drop_last=True)

In [5]:
specModel = DRSN_CBAM()
model_name = 'DRSN_CBAM_MAPE_Adam.pth'
model_name_ = 'DRSN_CBAM_MAPE_Adam'
TrainModel(specModel, model_name,model_name_)

-------第1轮训练开始-------
整体测试集上的loss:8559.24164666512
-------第2轮训练开始-------
整体测试集上的loss:4236.122851937836
-------第3轮训练开始-------
整体测试集上的loss:2855.031288505558
-------第4轮训练开始-------
整体测试集上的loss:2409.759006411347
-------第5轮训练开始-------
整体测试集上的loss:2111.594003656875
-------第6轮训练开始-------
整体测试集上的loss:1916.4787762848728
-------第7轮训练开始-------
整体测试集上的loss:1392.9807642021933
-------第8轮训练开始-------
整体测试集上的loss:1708.0300993980059
-------第9轮训练开始-------
整体测试集上的loss:1806.5526487040452
-------第10轮训练开始-------
整体测试集上的loss:766.9194253590848
-------第11轮训练开始-------
整体测试集上的loss:1307.7347826335754
-------第12轮训练开始-------
整体测试集上的loss:1688.7764404232173
-------第13轮训练开始-------
整体测试集上的loss:1782.4707356299773
-------第14轮训练开始-------
整体测试集上的loss:1443.9066828910293
-------第15轮训练开始-------
整体测试集上的loss:1311.2305059704909
-------第16轮训练开始-------
整体测试集上的loss:1510.0137051816225
-------第17轮训练开始-------
整体测试集上的loss:2114.754241094676
-------第18轮训练开始-------
整体测试集上的loss:1537.169286840903
-------第19轮训练开始-------
整体测试集上的loss:1183.093592

整体测试集上的loss:1597.8284270279155
-------第154轮训练开始-------
整体测试集上的loss:1586.589680184311
-------第155轮训练开始-------
整体测试集上的loss:1632.6509530107821
-------第156轮训练开始-------
整体测试集上的loss:1577.120921217413
-------第157轮训练开始-------
整体测试集上的loss:1537.444356048057
-------第158轮训练开始-------
整体测试集上的loss:1504.4673936205008
-------第159轮训练开始-------
整体测试集上的loss:1704.6215163455033
-------第160轮训练开始-------
整体测试集上的loss:1552.2055458569146
-------第161轮训练开始-------
整体测试集上的loss:1607.763638965161
-------第162轮训练开始-------
整体测试集上的loss:1572.977643183604
-------第163轮训练开始-------
整体测试集上的loss:1637.896628908051
-------第164轮训练开始-------
整体测试集上的loss:1589.4311376491287
-------第165轮训练开始-------
整体测试集上的loss:1636.4729919100105
-------第166轮训练开始-------
整体测试集上的loss:1667.733246632815
-------第167轮训练开始-------
整体测试集上的loss:1486.5277164342954
-------第168轮训练开始-------
整体测试集上的loss:1329.7783590885138
-------第169轮训练开始-------
整体测试集上的loss:1490.6625512211529
-------第170轮训练开始-------
整体测试集上的loss:1394.8332301759503
-------第171轮训练开始-------
整体测试集上的loss:1481.